In [ ]:
from MolRep import MolRep

## Construct Datasets and Dataloader

In [ ]:
basic_configs, dataset_configs, data, logger = MolRep.construct_training_data(
                                            dataset_name='BBBP', model_name='CMPNN', k_fold = 5)

## Training

In [ ]:
for fold_i in range(5):

    model_configs, model = MolRep.construct_model(
                        model_name = 'CMPNN',  model_config = 'MolRep/Configs/config_CMPNN.yml', 
                        basic_configs = basic_configs, dataset_configs = dataset_configs, logger = logger)

    # Optimizer
    optimizer = MolRep.construct_optimizer(model, model_configs)

    # Early_stopper
    stopper = MolRep.construct_stopper(model_configs)

    # Clipping
    clipping = MolRep.construct_clipping(model_configs)

    # Scheduler
    scheduler = MolRep.construct_scheduler(optimizer, model_configs, basic_configs)

    # Loss function
    loss_fn = MolRep.construct_loss_func(dataset_configs, basic_configs)

    # Trainer
    net = MolRep.construct_trainer(model, basic_configs, dataset_configs, model_configs, loss_fn)

    train_loader, test_loader, features_scaler, scaler = data[str(fold_i)]
    model, metric, loss = MolRep.training(net, train_loader, 
                                          test_loader=test_loader, 
                                          optimizer=optimizer, 
                                          scheduler=scheduler, 
                                          clipping=clipping, 
                                          stopper=stopper, 
                                          scaler=scaler, 
                                          logger=logger)
    MolRep.save_model('CMPNN_fold_{fold_i}.pt', model, scaler, features_scaler)